In [ ]:
import pandas as pd
import geopandas as gpd
import numpy as np
import os
from scipy.spatial.distance import pdist, squareform

%matplotlib inline

In [ ]:
parent_dir = os.path.split(os.getcwd())[0]

In [ ]:
zones = gpd.read_file(parent_dir + '\\Data\\New\\lms_zone_du_new.shp') # LMS Zone data

In [ ]:
density_data = zones[['ZONE_ID']]

Determine all the neccesary density variables and gather them in one dataframe. The population density and degree of urbanisation have already been determined before.

In [ ]:
density_data.loc[:, 'Pop_dens'] = np.round(zones.Pop_dens, 2) # people / ha
density_data.loc[:, 'Surrounding_pop_dens'] = np.round(zones.pop_dens_s, 2)
density_data.loc[:, 'DU'] = np.round(zones.deg_urba, 2)

density_data.loc[:, 'Job_dens'] = np.round(zones.BANENTOT / zones.OPP, 2) # jobs / ha

In [ ]:
density_data

Now determine the job density

In [ ]:
def job_dens_sur(gdf):
    """
    Determine the degree of urbanisation of the LMS zones
    """

    points = zones[['XCOORD', 'YCOORD']] # df with all coordinated centroids zone
    
    # Distances between all points
    distances = pd.DataFrame(squareform(pdist(points)), columns=points.index, index=points.index) 

    # All neigbour zones within a distance of 3 km
    close_zones = distances[(distances <= 3000) & (distances != 0)]

    # Create new columns
    gdf['Surrounding_job_dens'] = np.zeros((len(gdf)))


    # Get population density surrounding zones
    for i in close_zones.index:
        zone_list = close_zones.loc[i]

        index_list = list(map(tuple, np.argwhere(~np.isnan(zone_list.values))))

        sur_area = zones.loc[i]['OPP']
        sur_job = zones.loc[i]['BANENTOT']
        
        for j in range(len(index_list)):
            zone_id = index_list[j][0]

            sur_area += zones.loc[zone_id]['OPP']
            sur_job += zones.loc[zone_id]['BANENTOT']
        
        gdf.loc[i, 'Surrounding_job_dens'] = np.round(sur_job / sur_area, 2)
    
    return gdf

In [ ]:
density_data = job_dens_sur(density_data)

In [ ]:
density_data

## Save to csv

In [ ]:
density_data.to_csv(parent_dir + '\\Data\\New\\lms_zone_density.csv')